In [46]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")



In [4]:
dataset= load_dataset('sehyun66/Finnhub-News',split='train')
df = pd.DataFrame(dataset)

In [5]:
df = pd.DataFrame(dataset)

In [7]:
texts=list(df['headline'])

In [ ]:
inputs = tokenizer(texts, padding=True ,truncation=True,return_tensors="pt")


In [9]:
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
print(model.config.id2label[predicted_class_id])


In [10]:
device = torch.device('cuda')

In [ ]:
model.to(device)
model = torch.nn.DataParallel(model)
inputs.to(device)

In [39]:
# Define the chunk size
chunk_size = 1024  # Adjust this value as needed

# Initialize an empty list to store the results
results = []

# Loop through the input data in chunks
for i in tqdm(range(0, len(inputs['input_ids']), chunk_size)):
    chunk = inputs['input_ids'][i:i+chunk_size]
    chunk.to(device)
    # Perform inference on the chunk
    with torch.no_grad():
        output = model(chunk).logits   
    # Append the output to the results list
    results.append(output)

# Concatenate or process the results as needed
final_output = torch.cat(results, dim=0)  # Concatenate along the appropriate dimension


100%|██████████| 504/504 [04:37<00:00,  1.82it/s]


In [40]:
torch.save(final_output, 'sentiment_data/finbert.pt')

In [44]:
final_output[0]

tensor([-0.6055, -0.4091,  0.8146], device='cuda:0')

In [52]:
import numpy as np
df['sentiment'] = final_output.to('cpu').tolist()


In [67]:
df['sentiment_name'] = df['sentiment'].apply(lambda x:{'postive':x[0],'negative':x[1],'neutral':x[2]})

In [76]:
file_path = 'sentiment_data/finbert.json'
json_data = df.to_json(orient='records')
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

In [77]:
ds = load_dataset(
        "json",
        data_files={
            "train": "/home/ssu36/tiger/NH_competition/sentiment_data/*.json"
        },
    )

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 467.75it/s]
Generating train split: 515851 examples [00:11, 45993.29 examples/s]


In [78]:
ds.push_to_hub('sehyun66/Finnhub-News')

Creating parquet from Arrow format: 100%|██████████| 516/516 [00:01<00:00, 411.96ba/s]
